In [1]:
%%capture

import json
import speechbrain as sb
import os, sys
from speechbrain.utils.data_utils import get_all_files


In [2]:
def get_all_json_data(json_info, file_name="train"):
    data = {}
    for sess in json_info:
        if sess.endswith('.json'):
            wav_path = sess.replace('_metadata.json', '_mixture.wav')
        
            with open(sess) as f:
                json_data = json.load(f)
            
            json_data['wav_path'] = wav_path
            
            sess_id = os.path.basename(sess).split('_')[:-1] 
            sess_id = '_'.join(sess_id)
            
            json_data['num_speakers'] = sess_id.split("_")[-1]
            if sess_id not in data:
                data[sess_id] = []
            data[sess_id].append(json_data)
    
    # Save combined metadata to JSON
    metadata_path = os.path.join("../../data", f"{file_name}_data.json")
    with open(metadata_path, "w") as jsonfile:
        json.dump(data, jsonfile, indent=4)

In [3]:
train_json_info = get_all_files("../../data/train", match_and=['.json'])
dev_json_info = get_all_files("../../data/dev", match_and=['.json'])
eval_json_info = get_all_files("../../data/eval", match_and=['.json'])

get_all_json_data(train_json_info, file_name = "train")
get_all_json_data(dev_json_info, file_name = "dev")
get_all_json_data(eval_json_info, file_name = "eval")
print("Gathered all json data")

Gathered all json data


In [5]:
%%file hparams_crdnn_fbanks.yaml
# Seed and output folders
seed: 1986
__set_seed: !apply:torch.manual_seed [!ref <seed>]
output_folder: !ref results/ADSC_CRDNN/<seed>
save_folder: !ref <output_folder>/save
train_log: !ref <output_folder>/train_log.txt

# LibriParty (main data)
data_folder: ../../data/LibriParty  # e.g. /path/to/LibriParty

# Openrir Dataset for augmentation
data_folder_noise: !ref <data_folder>/noise # The noisy sequencies for data augmentation will automatically be downloaded here.
NOISE_DATASET_URL: https://www.dropbox.com/scl/fi/a09pj97s5ifan81dqhi4n/noises.zip?rlkey=j8b0n9kdjdr32o1f06t0cw5b7&dl=1
noise_csv_openrir: !ref <save_folder>/noise_openrir.csv #The data manifest files are created by the data preparation script

# Additional data (for augmentation)
#musan_folder: !PLACEHOLDER  # e.g, /path/to/musan (download it from the web before)
#commonlanguage_folder: !PLACEHOLDER  # e.g, /path/to/commonlang (download it from the web before)

# Manifest files (created by the data preparation)
annotation_train: !ref <save_folder>/train.json
annotation_valid: !ref <save_folder>/valid.json
annotation_test: !ref <save_folder>/test.json
#music_csv: !ref <save_folder>/music.csv
noise_csv: !ref <save_folder>/noise.csv
#speech_csv: !ref <save_folder>/speech.csv
#multilang_speech_csv: !ref <save_folder>/multilang_speech.csv
skip_prep: False # Skip data preparation

####################### Training Parameters ####################################
N_epochs: 100
lr: 1.0
lr_final: 0.1
batch_size: 2
example_length: 5 # in seconds
sample_rate: 16000
time_resolution: 0.01 # in seconds

num_workers: 4
train_dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: !ref <num_workers>
valid_dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: !ref <num_workers>
test_dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: !ref <num_workers>

# Feature parameters
n_fft: 400
n_mels: 40

####################### Model Parameters #######################################
# activation: !name:torch.nn.LeakyReLU
# dropout: 0.15
# cnn_blocks: 2
# cnn_channels: (16, 16)
# cnn_kernelsize: (3, 3)
rnn_layers: 2
rnn_neurons: 32
# rnn_bidirectional: True
# dnn_blocks: 1
dnn_neurons: 16
output_neurons: 1


# Data augmentation
# Download and prepare the dataset of noisy sequences for augmentation
prepare_noise_data: !name:speechbrain.augment.preparation.prepare_dataset_from_URL
    URL: !ref <NOISE_DATASET_URL>
    dest_folder: !ref <data_folder_noise>
    ext: wav
    csv_file: !ref <noise_csv_openrir>

add_noise: !new:speechbrain.augment.time_domain.AddNoise
    csv_file: !ref <noise_csv_openrir>
    snr_low: -5
    snr_high: 15
    noise_sample_rate: 16000
    clean_sample_rate: 16000
    num_workers: !ref <num_workers>

#add_noise_musan: !new:speechbrain.augment.time_domain.AddNoise
#    csv_file: !ref <noise_csv>
#    snr_low: -5
#    snr_high: 15
#    noise_sample_rate: 16000
#    clean_sample_rate: 16000
#    num_workers: !ref <num_workers>
#
#add_music_musan: !new:speechbrain.augment.time_domain.AddNoise
#    csv_file: !ref <music_csv>
#    snr_low: -5
#    snr_high: 15
#    noise_sample_rate: 16000
#    clean_sample_rate: 16000
#    num_workers: !ref <num_workers>
#
#add_speech_musan: !new:speechbrain.augment.time_domain.AddNoise
#    csv_file: !ref <speech_csv>
#    snr_low: -5
#    snr_high: 15
#    noise_sample_rate: 16000
#    clean_sample_rate: 16000
#    num_workers: !ref <num_workers>

#add_speech_multilang: !new:speechbrain.augment.time_domain.AddNoise
#    csv_file: !ref <multilang_speech_csv>
#    snr_low: -5
#    snr_high: 15
#    noise_sample_rate: 16000
#    clean_sample_rate: 16000
#    num_workers: !ref <num_workers>

# Models
compute_features: !new:speechbrain.lobes.features.Fbank
    sample_rate: !ref <sample_rate>
    n_fft: !ref <n_fft>
    n_mels: !ref <n_mels>
    hop_length: !ref <time_resolution> * 1000 # in ms

mean_var_norm: !new:speechbrain.processing.features.InputNormalization
    norm_type: sentence

cnn: !new:speechbrain.nnet.containers.Sequential
    input_shape: [null, null, !ref <n_mels>]
    norm1: !name:speechbrain.nnet.normalization.LayerNorm
    cnn1: !name:speechbrain.lobes.models.CRDNN.CNN_Block
        channels: 16
        kernel_size: (3, 3)
    cnn2: !name:speechbrain.lobes.models.CRDNN.CNN_Block
        channels: 32
        kernel_size: (3, 3)

rnn: !new:speechbrain.nnet.RNN.GRU
    input_shape: [null, null, 320]
    hidden_size: !ref <rnn_neurons>
    num_layers: !ref <rnn_layers>
    bidirectional: True

dnn: !new:speechbrain.nnet.containers.Sequential
    input_shape: [null, null, !ref <rnn_neurons> * 2]
    dnn1: !name:speechbrain.lobes.models.CRDNN.DNN_Block
        neurons: !ref <dnn_neurons>
    dnn2: !name:speechbrain.lobes.models.CRDNN.DNN_Block
        neurons: !ref <dnn_neurons>
    lin: !name:speechbrain.nnet.linear.Linear
        n_neurons: !ref <output_neurons>
        bias: False


model: !new:torch.nn.ModuleList
    - [!ref <cnn>, !ref <rnn>, !ref <dnn>]

modules:
    model: !ref <model>
    cnn: !ref <cnn>
    rnn: !ref <rnn>
    dnn: !ref <dnn>
    mean_var_norm: !ref <mean_var_norm>

opt_class: !name:torch.optim.Adadelta
    lr: !ref <lr>
    rho: 0.95
    eps: 1.e-8

epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <N_epochs>

lr_annealing: !new:speechbrain.nnet.schedulers.LinearScheduler
    initial_value: !ref <lr>
    final_value: !ref <lr_final>
    epoch_count: !ref <N_epochs>

checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        model: !ref <model>
        normalizer: !ref <mean_var_norm>
        counter: !ref <epoch_counter>

compute_BCE_cost: !name:speechbrain.nnet.losses.bce_loss

train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>

train_stats: !name:speechbrain.utils.metric_stats.BinaryMetricStats
test_stats: !name:speechbrain.utils.metric_stats.BinaryMetricStats

Overwriting hparams_crdnn_fbanks.yaml


In [ ]:
%%file train.py

import os
import sys
import torch
import torchaudio
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml
